# Stock selection and portfolio performance based on ESG Scores
by Chee-Foong on 11 Mar 2021

## Summary

This analysis studies the performance of portfolios constructed by selecting securities in the S&P 500 universe.  Stock selection are based on sectors and ESG scores.  The stocks in the portfolios are equally weighted and performance are compared with an equally weighted benchmark of all the securities in the S&P 500 universe.  

## Data

1. Yahoo Finance for financial information like prices, sectors and ESG scores
2. Datahub for list of securities in S&P 500

## Reference

1. https://datahub.io/core/s-and-p-500-companies#resource-constituents

---
## Import Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=Warning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np

# pd.options.display.max_columns = 50
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

plt.rcParams.update({'figure.figsize':(15,3), 'figure.dpi':60})
plt.style.use('fivethirtyeight')

In [2]:
!pip3 install edge

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import edge
import sys  
sys.path.append('../src') 
from edge import *
from edge_risk_kit import *
import edge_risk_kit as erk

In [4]:
# Others
import requests
import io
import string
import re
from pprint import pprint
from tqdm.notebook import tqdm
import io

In [11]:
DATA_FOLDER = 'C:/Users/20075008d/Desktop/data/'
createfolder(DATA_FOLDER)

Load list of S&P 500 constituents tickers

In [13]:
snp = pd.read_csv(DATA_FOLDER + '/constituents.csv')

In [14]:
snp.set_index('Symbol', inplace=True)

In [15]:
snp.head()

,Name,Sector
Symbol,,
MMM,3M,Industrials
AOS,A. O. Smith,Industrials
ABT,Abbott Laboratories,Health Care
ABBV,AbbVie,Health Care
ABMD,Abiomed,Health Care


## Data Extraction from Yahoo Finance
### ESG Scores

In [17]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.


In [18]:
import yfinance as yf

May take a while to extract information for all securities

In [19]:
# Retrieve Yahoo! Finance Sustainability Scores for each ticker
esg_data = pd.DataFrame([])

for ticker in tqdm(snp.index):

    try:
#         print('Processing {}'.format(ticker))
        temp = yf.Ticker(ticker).sustainability.T
        temp['symbol'] = ticker
        
        esg_data = pd.concat([esg_data,temp])
        
    except Exception as e:
#         print(e)
        continue
        
esg_data.set_index('symbol', inplace=True)

  0%|          | 0/505 [00:00<?, ?it/s]

- ANTM: No summary info found, symbol may be delisted
- BLL: No summary info found, symbol may be delisted
- CERN: No summary info found, symbol may be delisted
- CTXS: No summary info found, symbol may be delisted
- DISCA: No summary info found, symbol may be delisted
- DISCK: No summary info found, symbol may be delisted
- DRE: No summary info found, symbol may be delisted
- INFO: No summary info found, symbol may be delisted
- KSU: No summary info found, symbol may be delisted
- NLSN: No summary info found, symbol may be delisted
- NLOK: No summary info found, symbol may be delisted
- PBCT: No summary info found, symbol may be delisted
- VIAC: No summary info found, symbol may be delisted
- WLTW: No summary info found, symbol may be delisted
- XLNX: No summary info found, symbol may be delisted


Selecting required columns and cleaning up data

In [20]:
required_cols = ['socialScore','governanceScore','environmentScore','totalEsg',
                 'esgPerformance','percentile','peerGroup','highestControversy']

esg_data.columns.name = ''
esg_data = esg_data[required_cols]
esg_data = esg_data.apply(pd.to_numeric, errors='ignore') 

In [21]:
esg_data.shape

(435, 8)

In [22]:
esg_data.sort_values('totalEsg', ascending=False).head()

,socialScore,governanceScore,environmentScore,totalEsg,esgPerformance,percentile,peerGroup,highestControversy
symbol,,,,,,,,
CTRA,14.01,8.66,23.49,46.16,None,95.86,Oil & Gas Producers,2
OXY,12.04,6.59,24.61,43.23,None,93.00,Oil & Gas Producers,2
MRO,10.01,8.13,23.91,42.05,None,91.31,Oil & Gas Producers,2
GE,15.04,11.58,14.09,40.71,None,89.64,Industrial Conglomerates,3
TDG,17.85,8.29,12.84,38.98,LEAD_PERF,87.09,Aerospace & Defense,2


Save ESG Scores

In [23]:
esg_data.to_csv(DATA_FOLDER + 'esg_scores.csv')

---
### Security Reference

In [24]:
tickers = snp.index.to_list()

Extract Market Cap of all securities

In [25]:
tickerinfo = []
for ticker in tqdm(tickers):
    try:
        info = yf.Ticker(ticker).info
        info['symbol'] = ticker
        tickerinfo.append(info)
        
    except Exception as e:
        continue

  0%|          | 0/505 [00:00<?, ?it/s]

- ANTM: No summary info found, symbol may be delisted
- BLL: No summary info found, symbol may be delisted
- CERN: No summary info found, symbol may be delisted
- CTXS: No summary info found, symbol may be delisted
- DISCA: No summary info found, symbol may be delisted
- DISCK: No summary info found, symbol may be delisted
- DOV: No summary info found, symbol may be delisted
- DRE: No summary info found, symbol may be delisted
- EMR: No summary info found, symbol may be delisted
- EXR: No summary info found, symbol may be delisted
- INFO: No summary info found, symbol may be delisted
- KSU: No summary info found, symbol may be delisted
- NLSN: No summary info found, symbol may be delisted
- NLOK: No summary info found, symbol may be delisted
- PBCT: No summary info found, symbol may be delisted
- VIAC: No summary info found, symbol may be delisted
- WLTW: No summary info found, symbol may be delisted
- XLNX: No summary info found, symbol may be delisted


In [26]:
security_data = pd.DataFrame(tickerinfo)
security_data.set_index('symbol', inplace=True)

Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this already
Symbol removed from info (key='symbol'). You know this 

In [27]:
security_data.shape

(485, 146)

In [28]:
security_data.to_csv(DATA_FOLDER + 'security_data.csv')

---
### Prices

In [57]:
start_date = '2021-01-01'
end_date = '2022-12-31'

In [66]:
data = yf.download(tickers, start=start_date, end=end_date)

[*********************100%***********************]  505 of 505 completed

18 Failed downloads:
- NLOK: No timezone found, symbol may be delisted
- DRE: No timezone found, symbol may be delisted
- NLSN: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- FB: No timezone found, symbol may be delisted
- BRK.B: No timezone found, symbol may be delisted
- KSU: No timezone found, symbol may be delisted
- CERN: No timezone found, symbol may be delisted
- DISCA: No timezone found, symbol may be delisted
- DISCK: No timezone found, symbol may be delisted
- ANTM: No timezone found, symbol may be delisted
- BLL: No timezone found, symbol may be delisted
- PBCT: No timezone found, symbol may be delisted
- CTXS: No timezone found, symbol may be delisted
- WLTW: No timezone found, symbol may be delisted
- VIAC: No timezone found, symbol may be delisted
- INFO: No timezone found, symbol may be 

In [67]:
prices = data['Adj Close'][tickers]
prices

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY,CERN,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CPRT,GLW,CTVA,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISCA,DISCK,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,LLY,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FB,FAST,FRT,FDX,FIS,FITB,FRC,FE,FISV,FLT,FMC,F,FTNT,FTV,FBHS,FOXA,FOX,BEN,FCX,GPS,GRMN,IT,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HBI,HAS,HCA,PEAK,HSIC,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUM,HBAN,HII,IBM,IEX,IDXX,INFO,ITW,ILMN,INCY,IR,INTC,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,IPGP,IQV,IRM,JBHT,JKHY,J,SJM,JNJ,JCI,JPM,JNPR,KSU,K,KEY,KEYS,KMB,KIM,KMI,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LEG,LDOS,LEN,LNC,LIN,LYV,LKQ,LMT,L,LOW,LUMN,LYB,MTB,MRO,MPC,MKTX,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,TAP,MDLZ,MPWR,MNST,MCO,MS,MSI,MSCI,NDAQ,NTAP,NFLX,NWL,NEM,NWSA,NWS,NEE,NLSN,NKE,NI,NSC,NTRS,NOC,NLOK,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,OKE,ORCL,OGN,OTIS,PCAR,PKG,PH,PAYX,PAYC,PYPL,PENN,PNR,PBCT,PEP,PKI,PFE,PM,PSX,PNW,PXD,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PTC,PEG,PSA,PHM,PVH,QRVO,QCOM,PWR,DGX,RL,RJF,RTX,O,REG,REGN,RF,RSG,RMD,RHI,ROK,ROL,ROP,ROST,RCL,SPGI,CRM,SBAC,SLB,STX,SEE,SRE,NOW,SHW,SPG,SWKS,SNA,SO,LUV,SWK,SBUX,STT,STE,SYK,SIVB,SYF,SNPS,SYY,TMUS,TROW,TTWO,TPR,TGT,TEL,TDY,TFX,TER,TSLA,TXN,TXT,COO,HIG,HSY,MOS,TRV,DIS,TMO,TJX,TSCO,TT,TDG,TRMB,TFC,TWTR,TYL,TSN,USB,UDR,ULTA,UAA,UA,UNP,UAL,UPS,URI,UNH,UHS,VLO,VTR,VRSN,VRSK,VZ,VRTX,VFC,VIAC,VTRS,V,VNO,VMC,WRB,GWW,WAB,WBA,WMT,WM,WAT,WEC,WFC,WELL,WST,WDC,WU,WRK,WY,WHR,WMB,WLTW,WYNN,XEL,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-04 00:00:00-05:00,159.806458,51.779243,105.199554,95.901825,316.730011,248.901108,88.938156,47.784595,485.339996,149.533615,92.300003,21.922607,41.079159,117.230385,255.987839,105.239998,49.240002,150.416687,161.739441,526.460022,111.874298,47.171928,102.511505,86.306503,86.412003,35.128372,159.331497,10.508928,72.225700,15.13,76.104851,114.787941,35.418663,209.839798,145.578033,181.814926,93.868263,116.865356,213.126572,62.983284,140.956894,357.670013,NaN,201.070206,14.356004,127.874954,85.464371,129.509995,70.790001,117.237534,127.535355,18.542570,89.336784,296.839996,162.713135,1180.400024,146.124390,147.571686,19.736057,NaN,28.792398,31.166668,77.651894,237.989166,NaN,93.737274,575.260010,78.145012,242.949997,677.771118,39.047565,202.720001,2164.080078,36.872482,83.737495,35.270000,57.706631,400.089996,146.538147,45.614174,NaN,87.981606,135.759995,71.599998,45.213505,93.744995,49.498528,93.300003,20.379999,36.042690,103.730003,173.497482,92.610001,59.500000,124.464447,120.861862,62.090000,20.303865,102.529999,NaN,37.047531,249.669998,51.632835,647.030029,77.692520,1319.119995,144.735306,84.571373,198.795105,80.666695,337.010834,41.412872,55.554752,32.625397,NaN,188.075699,165.891327,56.297634,49.677780,77.226616,79.994736,47.963375,51.644150,33.239014,36.444527,65.528183,211.044998,60.325001,33.074291,37.522804,374.112549,14.343384,145.001862,28.6467

In [75]:
#px = prices.loc['2015-01-01':].dropna(axis=1, how='all')

In [76]:
#px.to_csv(DATA_FOLDER + 'prices.csv')
prices.to_csv(DATA_FOLDER + 'prices.csv')

In [93]:
prices

,MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BAC,BBWI,BAX,BDX,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CPRT,GLW,CTVA,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,LLY,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FAST,FRT,FDX,FIS,FITB,FRC,FE,FISV,FLT,FMC,F,FTNT,FTV,FBHS,FOXA,FOX,BEN,FCX,GPS,GRMN,IT,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HBI,HAS,HCA,PEAK,HSIC,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUM,HBAN,HII,IBM,IEX,IDXX,ITW,ILMN,INCY,IR,INTC,ICE,IFF,IP,IPG,INTU,ISRG,IVZ,IPGP,IQV,IRM,JBHT,JKHY,J,SJM,JNJ,JCI,JPM,JNPR,K,KEY,KEYS,KMB,KIM,KMI,KLAC,KHC,KR,LHX,LH,LRCX,LW,LVS,LEG,LDOS,LEN,LNC,LIN,LYV,LKQ,LMT,L,LOW,LUMN,LYB,MTB,MRO,MPC,MKTX,MAR,MMC,MLM,MAS,MA,MTCH,MKC,MCD,MCK,MDT,MRK,MET,MTD,MGM,MCHP,MU,MSFT,MAA,MRNA,MHK,TAP,MDLZ,MPWR,MNST,MCO,MS,MSI,MSCI,NDAQ,NTAP,NFLX,NWL,NEM,NWSA,NWS,NEE,NKE,NI,NSC,NTRS,NOC,NCLH,NRG,NUE,NVDA,NVR,NXPI,ORLY,OXY,ODFL,OMC,OKE,ORCL,OTIS,PCAR,PKG,PH,PAYX,PAYC,PYPL,PENN,PNR,PEP,PKI,PFE,PM,PSX,PNW,PXD,PNC,POOL,PPG,PPL,PFG,PG,PGR,PLD,PRU,PTC,PEG,PSA,PHM,PVH,QRVO,QCOM,PWR,DGX,RL,RJF,RTX,O,REG,REGN,RF,RSG,RMD,RHI,ROK,ROL,ROP,ROST,RCL,SPGI,CRM,SBAC,SLB,STX,SEE,SRE,NOW,SHW,SPG,SWKS,SNA,SO,LUV,SWK,SBUX,STT,STE,SYK,SIVB,SYF,SNPS,SYY,TMUS,TROW,TTWO,TPR,TGT,TEL,TDY,TFX,TER,TSLA,TXN,TXT,COO,HIG,HSY,MOS,TRV,DIS,TMO,TJX,TSCO,TT,TDG,TRMB,TFC,TYL,TSN,USB,UDR,ULTA,UAA,UA,UNP,UAL,UPS,URI,UNH,UHS,VLO,VTR,VRSN,VRSK,VZ,VRTX,VFC,VTRS,V,VNO,VMC,WRB,GWW,WAB,WBA,WMT,WM,WAT,WEC,WFC,WELL,WST,WDC,WU,WRK,WY,WHR,WMB,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-04 00:00:00-05:00,159.806458,51.779243,105.199554,95.901825,248.901108,88.938156,47.784595,485.339996,149.533615,92.300003,21.922607,41.079159,117.230385,255.987839,105.239998,49.240002,150.416687,161.739441,526.460022,111.874298,47.171928,102.511505,86.306503,86.412003,35.128372,159.331497,10.508928,72.225700,15.13,76.104851,114.787941,35.418663,209.839798,145.578033,181.814926,93.868263,116.865356,213.126572,62.983284,140.956894,357.670013,201.070206,14.356004,127.874954,85.464371,129.509995,70.790001,117.237534,127.535355,18.542570,89.336784,296.839996,162.713135,1180.400024,146.124390,147.571686,19.736057,28.792398,31.166668,77.651894,237.989166,93.737274,575.260010,78.145012,242.949997,677.771118,39.047565,202.720001,2164.080078,36.872482,83.737495,35.270000,57.706631,400.089996,146.538147,45.614174,87.981606,135.759995,71.599998,45.213505,93.744995,49.498528,93.300003,20.379999,36.042690,103.730003,173.497482,92.610001,59.500000,124.464447,120.861862,62.090000,20.303865,102.529999,37.047531,249.669998,51.632835,647.030029,77.692520,1319.119995,144.735306,84.571373,198.795105,80.666695,337.010834,41.412872,55.554752,32.625397,188.075699,165.891327,56.297634,49.677780,77.226616,79.994736,47.963375,51.644150,33.239014,36.444527,65.528183,211.044998,60.325001,33.074291,37.522804,374.112549,14.343384,145.001862,28.646784,210.551910,66.294922,65.974342,221.700470,108.656509,116.500000,261.473267,38.730000,52.608501,14.058197,89.434998,44.918228,126.137085,86.306915,31.530001

---
## Data Loading

In [119]:
esg_data = pd.read_csv(DATA_FOLDER + 'esg_scores.csv')
security_data = pd.read_csv(DATA_FOLDER + 'security_data.csv')
prices pd.read_csv(DATA_FOLDER + 'prices.csv')

In [120]:
esg_data.set_index('symbol', inplace=True)
security_data.set_index('symbol', inplace=True)
prices.Date = pd.to_datetime(prices.Date)
prices.set_index('Date', inplace=True)
prices.index = pd.to_datetime(prices.index, utc=True)
type(prices.index)

pandas.core.indexes.datetimes.DatetimeIndex

Remove securities that did not have enough price history

In [121]:
prices = prices[prices.columns[prices.count() == prices.count().max()]]

## Analytics Calculation

In [124]:
security_data.head()

,52WeekChange,SandP52WeekChange,address1,address2,algorithm,annualHoldingsTurnover,annualReportExpenseRatio,ask,askSize,averageDailyVolume3Month,beta,beta3Year,bid,bidSize,bookValue,category,circulatingSupply,city,coinMarketCapLink,companyOfficers,country,currentRatio,dateShortInterest,debtToEquity,dividendRate,dividendYield,earningsGrowth,earningsQuarterlyGrowth,ebitda,ebitdaMargins,enterpriseToEbitda,enterpriseToRevenue,enterpriseValue,exDividendDate,exchangeDataDelayedBy,exchangeName,expireDate,fax,financialCurrency,fiveYearAverageReturn,fiveYearAvgDividendYield,floatShares,forwardEps,forwardPE,freeCashflow,fromCurrency,fullTimeEmployees,fundFamily,fundInceptionDate,gmtOffSetMilliseconds,grossMargins,grossProfits,headSymbol,heldPercentInsiders,heldPercentInstitutions,impliedSharesOutstanding,industry,isEsgPopulated,lastCapGain,lastDividendDate,lastDividendValue,lastFiscalYearEnd,lastMarket,lastSplitDate,lastSplitFactor,legalType,logo_url,longBusinessSummary,longName,market,marketState,maxAge,maxSupply,messageBoardId,morningStarOverallRating,morningStarRiskRating,mostRecentQuarter,navPrice,netIncomeToCommon,nextFiscalYearEnd,numberOfAnalystOpinions,openInterest,operatingCashflow,operatingMargins,payoutRatio,pegRatio,phone,postMarketChange,postMarketPrice,preMarketChange,preMarketPrice,priceHint,priceToBook,priceToSalesTrailing12Months,profitMargins,quickRatio,quoteSourceName,quoteType,recommendationKey,recommendationMean,regularMarketChange,regularMarketSource,regularMarketTime,returnOnAssets,returnOnEquity,revenueGrowth,revenuePerShare,revenueQuarterlyGrowth,sector,sharesOutstanding,sharesPercentSharesOut,sharesShort,sharesShortPreviousMonthDate,sharesShortPriorMonth,shortName,shortPercentOfFloat,shortRatio,startDate,state,strikePrice,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,threeYearAverageReturn,toCurrency,totalAssets,totalCash,totalCashPerShare,totalDebt,totalRevenue,tradeable,trailingAnnualDividendRate,trailingAnnualDividendYield,trailingEps,trailingPE,trailingPegRatio,underlyingExchangeSymbol,underlyingSymbol,uuid,volume24Hr,volumeAllCurrencies,website,yield,ytdReturn,zip
symbol,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,3M Center,NaN,NaN,NaN,NaN,0.0,900.0,NaN,NaN,NaN,0.00,800.0,23.737,NaN,NaN,Saint Paul,NaN,[],United States,NaN,1.665706e+09,NaN,5.96,0.0503,NaN,NaN,NaN,0.00000,NaN,NaN,6.767535e+10,1.660867e+09,NaN,NaN,NaN,NaN,USD,NaN,3.24,NaN,10.59,11.094428,NaN,NaN,95000.0,NaN,NaN,-18000000.0,0.00000,1.657900e+10,NaN,0.00120,0.64990,NaN,Conglomerates,True,NaN,NaN,NaN,1.640909e+09,NaN,1.064880e+09,2:1,NaN,https://logo.clearbit.com/3m.com,"3M Company operates as a diversified technology company worldwide. It operates through four segments: Safety and Industrial; Transportation and Electronics; Health Care; and Consumer. The Safety and Industrial segment offers industrial abrasives and finishing for metalworking applications; autobody repair solutions; closure systems for personal hygiene products, masking, and packaging materials; electrical products and materials for construction and maintenance, power distribution, and electrical original equipment manufacturers; structural adhesives and tapes; respiratory, hearing, eye, and fall protection solutions; and natural and color-coated mineral granules for shingles. The Transportation and Electronics segment provides ceramic solutions; attachment tapes, films, sound, and temperature management for transportation vehicles; premium large format graphic films for advertising and fleet signage; light management films and electronics assembly solutions; packaging and interconnection solutions; and reflective signage for highway, and vehicle safety. The Healthcare segment offers food safety indicator solutions; health care procedure coding and reimbursement software; skin, wound care, and infection prevention products and solutions; dentistry 

In [122]:
def calcRetsCov(prices, freq):
    prices_freq = prices.resample(freq).fillna('ffill')
    prices_freq.index = prices_freq.index.to_period(freq)

    rets = prices_freq.pct_change().dropna(axis=1, how='all').dropna()
    cov = rets.cov()
    return rets, cov

rets_monthly, cov_monthly = calcRetsCov(prices, 'M')
rets_period = rets_monthly
PERIODS_PER_YEAR = 12
RISK_FREE_RATE = 0.013
risk_data = erk.summary_stats(rets_period, riskfree_rate=RISK_FREE_RATE, periods_per_year=PERIODS_PER_YEAR).sort_values('Sharpe Ratio', ascending=False)

full_data = risk_data.join(esg_data).join(security_data['marketCap']).join(snp)
full_data = full_data[~full_data.totalEsg.isnull()]
full_data['mktcap_grp'] = pd.cut(full_data.percentile, 3, labels=["Small", "Medium", "Large"])

largePeerGroup = esg_data.peerGroup.value_counts().index[0:20].to_list()
full_data['peerGroup2'] = full_data.peerGroup.apply(lambda x: x if x in largePeerGroup else 'Others')

full_data['esg_soc_grp'] = pd.cut(full_data.socialScore, 5, labels=["Severe Risk", "High Risk", "Medium Risk", "Low Risk", "No Risk"])
full_data['esg_env_grp'] = pd.cut(full_data.environmentScore, 5, labels=["Severe Risk", "High Risk", "Medium Risk", "Low Risk", "No Risk"])
full_data['esg_gov_grp'] = pd.cut(full_data.governanceScore, 5, labels=["Severe Risk", "High Risk", "Medium Risk", "Low Risk", "No Risk"])
full_data['esg_tot_grp'] = pd.cut(full_data.totalEsg, 5, labels=["Severe Risk", "High Risk", "Medium Risk", "Low Risk", "No Risk"])

KeyError: 'marketCap'

There are 2 columns classifying the securities to respective industry: Sector and peerGroup.  Sector comes from the security reference information downloaded from datahub and peerGroup from the ESG data downloaded from yahoo finance.  peerGroup data is more granular.  For analysis purpose, we will select the top 20 peerGroup with the most securities and the rest to be grouped under Others.  peerGroup2 column is created to store this data.

---
## Data Exploratory and Analytics

In [ ]:
full_data.head()

In [ ]:
full_data.shape

There are 449 securities for selection for this analysis.

## Pair Scatter Plots of ESG Scores

In [ ]:
sns.pairplot(data = full_data[['socialScore','environmentScore','governanceScore',
                               'Annualized Return','Annualized Vol','Sector']], hue='Sector')
plt.show()

#### Observations
1. From this histograms, companies in Real Estate have rather low social, environment and goverance score.
2. Most companies have rather low environment score.  This is especially true of companies in Health Care, Industrial and Financials.  This could be a case that these companies are operating without much environmental concerns to address in the first place.
3. Note that companies in Materials and Energy have high environment scores.  These companies are likely to focus on addressing this concerns due to their nature of work.
4. All companies have some form of governance and hence there are no zero governance score.  However, many companies have low governance scores.  Possible that companies pay more attention to social and environment concerns that are more obvious to outsiders.  Governance scoring requires more information from insiders like staffs and suppliers.
5. It is rather obvious that Financial, Energy, Health Care sectors have highest governance, environment and social scores generating average annualised returns.  Similar observations can be found for annualised volatility, especially Financials and Health Care.

In [ ]:
full_data.Sector.value_counts().plot.bar(figsize=(15,3))
plt.show()

In [ ]:
print('5 sectors with most securities: {}'.format(', '.join(full_data.Sector.value_counts().head().index.to_list())))

In [ ]:
full_data.peerGroup2.value_counts().plot.bar(figsize=(15,3))
plt.show()

In [ ]:
print('5 peerGroup with most securities: {}'.format(', '.join(full_data.peerGroup2.value_counts()[1:6].index.to_list())))

It is good to keep in mind the number of data points in each Sectors and Peer Groups as we dissect the ESG below.

### Total ESG

In [ ]:
def sorted_boxplot(data, x, y):
    grouped = data[[x,y]].groupby([x]).median().sort_values(y, ascending=False)
    ax = sns.boxplot(data=data, x=y, y=x, order=grouped.index, orient='h')
    ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
    plt.show()

In [ ]:
sorted_boxplot(full_data, 'Sector', 'totalEsg')

### Environment Score

In [ ]:
sorted_boxplot(full_data, 'Sector', 'environmentScore')

### Governance Score

In [ ]:
sorted_boxplot(full_data, 'Sector', 'governanceScore')

### Social Score

In [ ]:
sorted_boxplot(full_data, 'Sector', 'socialScore')

#### Observations
1. Securities in energy-related sectors tends to have high environment ESG Score.  Relatively, they are higher than those securities with highest goverance and social ESG Score.  Hence, one can observe that the securities with high total ESG scores are from also from energy-related sectors.  The top 5 sectors with highest total ESG scores and the top 5 sectors with highest environement ESG scores are the same.
2. We can expect the performance of an equally weighted portfolio to be the same if we select the securities based on total ESG or environment ESG score.

---
## Portfolio Analysis

For this analysis, we create an equally weighted portfolio of all securities across all industries as our benchmark.  Performance of portfolios based on various security selection criterias are compared against this benchmark.  For consistency, each portfolio will only have 20 securities out of the total securities in the universe.

### Stock Selection 1
1. Sector = All
2. Number of portfolios = 4
3. Stock selection = Top 30 securities in terms of social, governance, environment, total ESG Score
4. Portfolio Names: 
    - socialScore
    - governanceScore
    - environmentScore
    - totalEsg

In [ ]:
score_list = ['socialScore','governanceScore','environmentScore','totalEsg']
num_of_stocks = 30

## only need the expected return to generate the equal weights... other than that is redundant for now
er_bmk = erk.annualize_rets(rets_monthly, 12)
cov_bmk = rets_monthly.cov()

return_bmk = (ew(er_bmk) * rets_monthly).sum(axis=1)
wealth_bmk = erk.drawdown(return_bmk).Wealth 

return_port = {}
wealth_port = {}

return_port['bmk'] = return_bmk
wealth_port['bmk'] = wealth_bmk

for score in score_list:
    return_ = {}
    wealth_ = {}
    
    stock_selected = full_data.sort_values(score, ascending=False).head(num_of_stocks).index

    ## only need the expected return to generate the equal weights... other than that is redundant for now
    er_port = erk.annualize_rets(rets_monthly[stock_selected], 12)
    cov_port = rets_monthly[stock_selected].cov()
    
    return_ = (ew(er_port) * rets_monthly[stock_selected]).sum(axis=1)
    wealth_ = erk.drawdown(return_).Wealth 

    return_port[score] = return_
    wealth_port[score] = wealth_
    
return_port = pd.DataFrame(return_port)
wealth_port = pd.DataFrame(wealth_port)    

return_port.plot(title='Portfolio Monthly Return').legend(bbox_to_anchor=(1.0, 1.0))
plt.show()

wealth_port.plot(title='Cumulative Performance').legend(bbox_to_anchor=(1.0, 1.0))
plt.show()

erk.summary_stats(return_port, riskfree_rate=RISK_FREE_RATE, periods_per_year=PERIODS_PER_YEAR).sort_values('Sharpe Ratio', ascending=False)    

In [ ]:
# full_data.loc[stock_selected.to_list()]

#### Observations
1. socialScore portfolio outperformed the benchmark.  This portfolio has the highest sharpe ratio at 0.856.
2. environmentScore and totalEsg portfolios' performances are poor.  totalEsg at the bottom with a sharp ratio of only 0.209.
3. Blinding selecting securities based on good totalEsg underperforms the benchmark significantly.
4. Good social Score securities come from good performing sectors (Health Care, Industrial, Financials) that outperforms the benchmark.

### Stock Selection 2

***Assuming you selected the top 3 outperforming sectors correctly***

1. Sector = 'Information Technology', 'Communication Services', 'Health Care'
2. Number of portfolios = 4
3. Stock selection = Top 30 securities in terms of social, governance, environment, total ESG Score
4. Portfolio Names: 
    - socialScore
    - governanceScore
    - environmentScore
    - totalEsg

In [ ]:
sorted_boxplot(full_data, 'Sector', 'Annualized Return')

In [ ]:
sector_list = ['Information Technology', 'Communication Services', 'Health Care']
score_list = ['socialScore','governanceScore','environmentScore','totalEsg']
num_of_stocks = 30

## only need the expected return to generate the equal weights... other than that is redundant for now
er_bmk = erk.annualize_rets(rets_monthly, 12)
cov_bmk = rets_monthly.cov()

return_bmk = (ew(er_bmk) * rets_monthly).sum(axis=1)
wealth_bmk = erk.drawdown(return_bmk).Wealth 

return_port = {}
wealth_port = {}

return_port['bmk'] = return_bmk
wealth_port['bmk'] = wealth_bmk

for score in score_list:
    return_ = {}
    wealth_ = {}
    
    stock_selected = full_data[full_data.Sector.isin(sector_list)].sort_values(score, ascending=False).head(num_of_stocks).index
    
    ## only need the expected return to generate the equal weights... other than that is redundant for now
    er_port = erk.annualize_rets(rets_monthly[stock_selected], 12)
    cov_port = rets_monthly[stock_selected].cov()
    
    return_ = (ew(er_port) * rets_monthly[stock_selected]).sum(axis=1)
    wealth_ = erk.drawdown(return_).Wealth 

    return_port[score] = return_
    wealth_port[score] = wealth_
    
return_port = pd.DataFrame(return_port)
wealth_port = pd.DataFrame(wealth_port)    

return_port.plot(title='Portfolio Monthly Return').legend(bbox_to_anchor=(1.0, 1.0))
plt.show()

wealth_port.plot(title='Cumulative Performance').legend(bbox_to_anchor=(1.0, 1.0))
plt.show()

erk.summary_stats(return_port, riskfree_rate=RISK_FREE_RATE, periods_per_year=PERIODS_PER_YEAR).sort_values('Sharpe Ratio', ascending=False)    

In [ ]:
print('Stocks Selected: {}'.format(', '.join(full_data.loc[stock_selected.to_list()].Name.to_list())))

#### Observations

1. It is quite expected that all the portfolios will outperforms the benchmark.
2. If sector picking is good, picking top 30 stocks with highest total ESG score will return a sharpe ratio of 1.18.
3. Note that even through companies in the Energy sectors tends to have high environment score and total ESG score, the portfolio performances are not due to these companies as Energy is not one of the select sectors to invest in. 


### Stock Selection 3

***Assuming you eliminated the top 3 underperforming sectors correctly***

1. Sector Excluded = 'Utilities', 'Real Estate', 'Energy'
2. Number of portfolios = 4
3. Stock selection = Top 30 securities in terms of social, governance, environment, total ESG Score
4. Portfolio Names: 
    - socialScore
    - governanceScore
    - environmentScore
    - totalEsg

In [ ]:
sorted_boxplot(full_data, 'Sector', 'Annualized Return')

In [ ]:
sector_list = [s for s in list(set(full_data.Sector)) if s not in ['Utilities', 'Real Estate', 'Energy']]
score_list = ['socialScore','governanceScore','environmentScore','totalEsg']
num_of_stocks = 30

## only need the expected return to generate the equal weights... other than that is redundant for now
er_bmk = erk.annualize_rets(rets_monthly, 12)
cov_bmk = rets_monthly.cov()

return_bmk = (ew(er_bmk) * rets_monthly).sum(axis=1)
wealth_bmk = erk.drawdown(return_bmk).Wealth 

return_port = {}
wealth_port = {}

return_port['bmk'] = return_bmk
wealth_port['bmk'] = wealth_bmk

for score in score_list:
    return_ = {}
    wealth_ = {}
    
    stock_selected = full_data[full_data.Sector.isin(sector_list)].sort_values(score, ascending=False).head(num_of_stocks).index
    
    ## only need the expected return to generate the equal weights... other than that is redundant for now
    er_port = erk.annualize_rets(rets_monthly[stock_selected], 12)
    cov_port = rets_monthly[stock_selected].cov()
    
    return_ = (ew(er_port) * rets_monthly[stock_selected]).sum(axis=1)
    wealth_ = erk.drawdown(return_).Wealth 

    return_port[score] = return_
    wealth_port[score] = wealth_
    
return_port = pd.DataFrame(return_port)
wealth_port = pd.DataFrame(wealth_port)    

return_port.plot(title='Portfolio Monthly Return').legend(bbox_to_anchor=(1.0, 1.0))
plt.show()

wealth_port.plot(title='Cumulative Performance').legend(bbox_to_anchor=(1.0, 1.0))
plt.show()

erk.summary_stats(return_port, riskfree_rate=RISK_FREE_RATE, periods_per_year=PERIODS_PER_YEAR).sort_values('Sharpe Ratio', ascending=False)    

#### Observation
1.  The portfolio with the top Social score performs the best.  However, it only beats the benckmark slightly.
2.  There is no strong reason to believe the portfolio construction based on ESG scores alone will outperform the benchmark.

### Stock Selection 4

***A native sector selection based on observation from the pairs plot.  Financials and Health Care have high governance and social score with average annualised returns and volatility.***

1. Sector = 'Financials', 'Health Care'
2. Number of portfolios = 4
3. Stock selection = Top 30 securities in terms of social, governance, environment, total ESG Score
4. Portfolio Names: 
    - socialScore
    - governanceScore
    - environmentScore
    - totalEsg

In [ ]:
sector_list = ['Financials', 'Health Care']
score_list = ['socialScore','governanceScore','environmentScore','totalEsg']
num_of_stocks = 30

## only need the expected return to generate the equal weights... other than that is redundant for now
er_bmk = erk.annualize_rets(rets_monthly, 12)
cov_bmk = rets_monthly.cov()

return_bmk = (ew(er_bmk) * rets_monthly).sum(axis=1)
wealth_bmk = erk.drawdown(return_bmk).Wealth 

return_port = {}
wealth_port = {}

return_port['bmk'] = return_bmk
wealth_port['bmk'] = wealth_bmk

for score in score_list:
    return_ = {}
    wealth_ = {}
    
    stock_selected = full_data[full_data.Sector.isin(sector_list)].sort_values(score, ascending=False).head(num_of_stocks).index
    
    ## only need the expected return to generate the equal weights... other than that is redundant for now
    er_port = erk.annualize_rets(rets_monthly[stock_selected], 12)
    cov_port = rets_monthly[stock_selected].cov()
    
    return_ = (ew(er_port) * rets_monthly[stock_selected]).sum(axis=1)
    wealth_ = erk.drawdown(return_).Wealth 

    return_port[score] = return_
    wealth_port[score] = wealth_
    
return_port = pd.DataFrame(return_port)
wealth_port = pd.DataFrame(wealth_port)    

return_port.plot(title='Portfolio Monthly Return').legend(bbox_to_anchor=(1.0, 1.0))
plt.show()

wealth_port.plot(title='Cumulative Performance').legend(bbox_to_anchor=(1.0, 1.0))
plt.show()

erk.summary_stats(return_port, riskfree_rate=RISK_FREE_RATE, periods_per_year=PERIODS_PER_YEAR).sort_values('Sharpe Ratio', ascending=False)    

#### Observation
1. Surprisingly, the portfolio with 30 stocks with the best total ESG score in the Financials and Health Care sectors performs well.  It returns a Sharpe Ratio of 0.926.  Slightly higher volatility then the benchmark but lower in maximum drawdown.
2. The portfolio constructed based on only the governance score is too volatile and has a much lower Sharpe Ratio then the benchmark.